In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_google_genai import ChatGoogleGenerativeAI


In [2]:
# Importing the env variables
load_dotenv()

True

In [3]:
# Model
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [21]:
# Defining prompt template for different types

positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helful assitant"),
        ("human", "Generate a thank you note for the reviewer who gave a positive feedback: {feedback}")
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assitant"),
        ("human","Generate a response addresing the reviewer who gave a negative feedback : {feedback}")
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assitant"),
        ("human", "Generate a request for more details for the reviewer who gave a neutral feedback : {feedback}")
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful agent"),
        ("human","Generate a message the reviewer that we will escalate this feedback to human agent : {feedback}")
    ]
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful agent"),
        ("human","Classify the sentiment of this feedback as positive, negative, neutral or esclate : {feedback}")
    ]
)


In [22]:
# DEfineing the branches
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | model | StrOutputParser()
    ),
        escalate_feedback_template | model | StrOutputParser()
)

In [23]:
#classificagion_chian

classification_chain = classification_template | model | StrOutputParser()

final_chain = classification_chain | branches

In [24]:
# Run the chain with an example review
good_review = "The product is excellent. I really enjoyed using it and found it very helpful."
bad_review = "The product is terrible. It broke after just one use and the quality is very poor."
neutral_review = "The product is okay. It works as expected but nothing exceptional."
default = "I'm not sure about the product yet. Can you tell me more about its features and benefits?"



In [25]:
result = final_chain.invoke({"feedback":bad_review})
print(result)

Okay, here are a few options for a message to the reviewer indicating that their negative feedback will be escalated to a human agent, with varying levels of formality:

**Option 1 (Neutral & Concise):**

> Thank you for your feedback. We are escalating this issue to a human agent for further review.

**Option 2 (Slightly More Empathetic):**

> We appreciate you bringing this to our attention. We're escalating your feedback to a human agent who can investigate further.

**Option 3 (Formal):**

> Thank you for providing your feedback. Due to the nature of your concern, we are escalating this matter to a human agent for review and appropriate action.

**Option 4 (Focus on Resolution):**

> We understand your concern. To ensure this is addressed properly, we are escalating your feedback to a human agent who will be able to assist you further.

**Option 5 (If you need more information):**

> Thank you for your feedback. We are escalating this to a human agent for further review. They may c